In [ ]:
"""Notebook for the extraction of demographic and clinical variables from ADNI. We use ADNI merge dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import xml.etree.cElementTree as et

In [ ]:
#Get list of MCI

In [2]:
subjectlist=os.listdir("/local_mount/space/celer/1/users/notebooks/moises/pdata/newdata/mcisplit/whole")

In [ ]:
#Get all PET dates from metadata in original ADNI folder

In [4]:
dates=[]
for i in subjectlist:
    key=i[11:-4]
    #print(key)
    for element in os.listdir("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/"):
        if key in element:
            #print("Got the key!",element)
            tree=et.parse(os.path.join("/local_mount/space/celer/1/users/david/Datasets/new_ADNI/ADNI/",element))
            raiz=tree.getroot()
            for group in raiz.iter("dateAcquired"):
                a=group.text
                #print(i,a)
                dates.append(a)
            break
    

In [ ]:
#Create object using ID and date, then we will check for presence in Merge and filter by this to get only data from
#the patients in our dataset

In [5]:
subjectlist[0][:10]+dates[0][:4]

'109_S_11142007'

In [6]:
presence={}
for i,j in enumerate(subjectlist):
    niu=j[:10]+dates[i][:4]
    presence.update({niu:1})
    

In [7]:
pr=pd.DataFrame.from_dict(presence, orient='index',columns=['pres'])

In [8]:
pr

,pres
109_S_11142007,1
033_S_09062006,1
012_S_10332006,1
035_S_09972007,1
109_S_62212019,1
...,...
137_S_06692006,1
037_S_61252018,1
073_S_09092006,1
041_S_14202007,1


In [9]:
cd /local_mount/space/celer/1/users/notebooks/moises

/local_mount/space/celer/1/users/notebooks/moises


In [10]:
merge=pd.read_csv('ADNIMERGE.csv')

/autofs/space/celer_001/users/software/miniconda3/envs/DL/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
a=list(merge['PTID'])
b=list(merge['EXAMDATE'])
newthing={}
for i,j in enumerate(a):
    c=j+b[i][:4]
    newthing.update({i:c})  

In [13]:
nw=pd.DataFrame.from_dict(newthing,orient='index', columns=['newid'])

In [14]:
mergy=pd.concat((merge,nw),axis=1)

In [16]:
mergy=mergy.set_index('newid')

In [ ]:
#This line gets the first entry in the year in case there are two

In [17]:
mergy = mergy[~mergy.index.duplicated(keep='first')]

In [18]:
demo=pd.concat((mergy,pr),axis=1)

In [ ]:
#And here we drop the values from patients not in our dataset

In [20]:
demo=demo.dropna(subset=['pres'])

In [23]:
demo.describe()

,RID,SITE,AGE,PTEDUCAT,APOE4,FDG,PIB,AV45,FBB,CDRSB,...,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,pres
count,364.000000,364.000000,362.000000,364.000000,327.000000,309.000000,13.000000,89.000000,95.000000,344.000000,...,159.000000,299.000000,12.000000,62.000000,96.000000,364.000000,364.000000,364.000000,364.000000,373.0
mean,3422.585165,88.664835,73.480110,15.961538,0.587156,1.236601,1.684231,1.265378,1.178867,1.734012,...,1.877628,1.241241,1.640833,1.259440,1.177442,0.793397,9.501263,9.494505,9.280220,1.0
std,2907.639265,131.699519,7.464942,2.715866,0.672113,0.081093,0.363787,0.269906,0.243236,1.386537,...,0.667444,0.074052,0.343020,0.278199,0.242356,1.839151,22.024612,22.040970,21.645237,0.0
min,33.000000,2.000000,55.100000,8.000000,0.000000,0.933673,1.195000,0.810000,0.870200,0.000000,...,1.000000,0.933673,1.195000,0.810000,0.870200,0.000000,0.000000,0.000000,0.000000,1.0
25%,727.500000,19.000000,69.000000,14.000000,0.000000,1.191070,1.362500,1.016000,0.991400,1.000000,...,1.293385,1.194900,1.361875,1.017525,0.991400,0.000000,0.000000,0.000000,0.000000,1.0
50%,1354.500000,41.000000,73.900000,16.000000,0.000000,1.250350,1.567500,1.316100,1.057300,1.500000,...,1.710530,1.251110,1.528750,1.209200,1.055650,0.000000,0.000000,0.000000,0.000000,1.0
75%,6619.750000,127.250000,78.900000,18.000000,1.000000,1.293780,1.902500,1.517700,1.385350,2.125000,...,2.338095,1.293440,1.851875,1.529925,1.380525,0.000000,0.000000,0.000000,0.000000,1.0
max,6964.000000,941.000000,90.000000,20.000000,2.000000,1.416430,2.282500,1.802200,1.816000,13.000000,...,3.685710,1.416430,2.282500,1.802200,1.816000,12.646100,151.443000,150.000000,150.000000,1.0


In [154]:
a=list(demo.columns)

In [155]:
a

['RID',
 'COLPROT',
 'ORIGPROT',
 'PTID',
 'SITE',
 'VISCODE',
 'EXAMDATE',
 'DX_bl',
 'AGE',
 'PTGENDER',
 'PTEDUCAT',
 'PTETHCAT',
 'PTRACCAT',
 'PTMARRY',
 'APOE4',
 'FDG',
 'PIB',
 'AV45',
 'FBB',
 'ABETA',
 'TAU',
 'PTAU',
 'CDRSB',
 'ADAS11',
 'ADAS13',
 'ADASQ4',
 'MMSE',
 'RAVLT_immediate',
 'RAVLT_learning',
 'RAVLT_forgetting',
 'RAVLT_perc_forgetting',
 'LDELTOTAL',
 'DIGITSCOR',
 'TRABSCOR',
 'FAQ',
 'MOCA',
 'EcogPtMem',
 'EcogPtLang',
 'EcogPtVisspat',
 'EcogPtPlan',
 'EcogPtOrgan',
 'EcogPtDivatt',
 'EcogPtTotal',
 'EcogSPMem',
 'EcogSPLang',
 'EcogSPVisspat',
 'EcogSPPlan',
 'EcogSPOrgan',
 'EcogSPDivatt',
 'EcogSPTotal',
 'FLDSTRENG',
 'FSVERSION',
 'IMAGEUID',
 'Ventricles',
 'Hippocampus',
 'WholeBrain',
 'Entorhinal',
 'Fusiform',
 'MidTemp',
 'ICV',
 'DX',
 'mPACCdigit',
 'mPACCtrailsB',
 'EXAMDATE_bl',
 'CDRSB_bl',
 'ADAS11_bl',
 'ADAS13_bl',
 'ADASQ4_bl',
 'MMSE_bl',
 'RAVLT_immediate_bl',
 'RAVLT_learning_bl',
 'RAVLT_forgetting_bl',
 'RAVLT_perc_forgetting_bl',

In [ ]:
#Transform back from ID+Date to just the ID

In [30]:
newindex=[]
for i in demo.index:
    i=i[:10]
    newindex.append(i)

In [34]:
t=list(demo['PTGENDER'])
y=list(demo['AGE'])
u=list(demo['ORIGPROT'])
p=list(demo['PTEDUCAT'])
h=list(demo['APOE4'])
j=list(demo['MMSE'])
k=list(demo['MOCA'])

In [ ]:
#Create final dataset with only columns of interest, the index is the patient ID

In [35]:
demog=pd.DataFrame(list(zip(t, y,u,p,h,j,k)),
               columns =['Gender', 'Age','Phase','Education_years','APOE4','MMSE_dem','MOCA'],
                   index=newindex)

In [39]:
pwd

'/local_mount/space/celer/1/users/notebooks/moises'

In [40]:
demog.to_excel('processed_demog.xlsx')